In [1]:
import pandas as pd
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)
from sklearn.preprocessing import OneHotEncoder
import plotly.figure_factory as ff
import psycopg2
import plotly.express as px
import pandas.io.sql as sqlio
import plotly.figure_factory as ff
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
from datetime import datetime
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [2]:
conn = psycopg2.connect(dbname="postgres", user="admin_film", password="111", host="127.0.0.1", port="5432")
print("Подключение установлено")

Подключение установлено


In [4]:
sql = """with cus_add as (
select customer_id,amount,city,district,s.last_name,film_id,inventory_id
from address 
join city using(city_id)
join customer as cu using(address_id)
join inventory using(store_id)
join payment as p using(customer_id)
join staff as s using(staff_id)
)

select r.customer_id,amount,city,district,
c.name as category,length,rating,title,
extract(day from (return_date - rental_date)) as rental_period,
cad.last_name,l.name,release_year,special_features
from (select * from cus_add limit 100000) as cad
join film as f using(film_id)
join rental as r using(inventory_id)
join film_category as fc using(film_id)
join category as c using(category_id)
join language as l using(language_id)"""
df = pd.read_sql_query(sql, conn)
df.head()

,customer_id,amount,city,district,category,length,rating,title,rental_period,last_name,name,release_year,special_features
0,269,2.99,Cheju,Cheju,Horror,123,PG-13,SWARM GOLD,4.0,Stephens,English,2006,"[Trailers, Commentaries]"
1,269,1.99,Araatuba,So Paulo,Horror,123,PG-13,SWARM GOLD,4.0,Hillyer,English,2006,"[Trailers, Commentaries]"
2,269,7.99,Rockford,Illinois,Horror,123,PG-13,SWARM GOLD,4.0,Stephens,English,2006,"[Trailers, Commentaries]"
3,269,5.99,La Romana,La Romana,Horror,123,PG-13,SWARM GOLD,4.0,Hillyer,English,2006,"[Trailers, Commentaries]"
4,269,2.99,La Romana,La Romana,Horror,123,PG-13,SWARM GOLD,4.0,Hillyer,English,2006,"[Trailers, Commentaries]"


In [5]:
df = df.dropna()

In [6]:
df

,customer_id,amount,city,district,category,length,rating,title,rental_period,last_name,name,release_year,special_features
0,269,2.99,Cheju,Cheju,Horror,123,PG-13,SWARM GOLD,4.0,Stephens,English,2006,"[Trailers, Commentaries]"
1,269,1.99,Araatuba,So Paulo,Horror,123,PG-13,SWARM GOLD,4.0,Hillyer,English,2006,"[Trailers, Commentaries]"
2,269,7.99,Rockford,Illinois,Horror,123,PG-13,SWARM GOLD,4.0,Stephens,English,2006,"[Trailers, Commentaries]"
3,269,5.99,La Romana,La Romana,Horror,123,PG-13,SWARM GOLD,4.0,Hillyer,English,2006,"[Trailers, Commentaries]"
4,269,2.99,La Romana,La Romana,Horror,123,PG-13,SWARM GOLD,4.0,Hillyer,English,2006,"[Trailers, Commentaries]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
349788,393,2.99,La Romana,La Romana,Drama,105,G,MOB DUFFEL,6.0,Hillyer,English,2006,[Trailers]
349789,393,5.99,La Romana,La Romana,Drama,105,G,MOB DUFFEL,6.0,Hillyer,English,2006,[Trailers]
349790,393,7.99,Rockford,Illinois,Drama,105,G,MOB DUFFEL,6.0,Stephens,English,2006,[Trailers]
349791,393,2.99,Cheju,Cheju,Drama,105,G,MOB DUFFEL,6.0,Stephens,English,2006,[Trailers]


In [9]:
df['special_features'] = df['special_features'].astype('str')

In [25]:
df['length'] = df['length'].astype('int')

In [10]:
city = df['city'].unique()
district = df['district'].unique()
category = df['category'].unique()

rating = df['rating'].unique()
title = df['title'].unique()
last_name = df['last_name'].unique()
name = df['name'].unique()
release_year = df['release_year'].unique()
special_features = df['special_features'].unique()

In [56]:
df['special_features'].unique().tolist()

["['Trailers', 'Commentaries']",
 "['Commentaries', 'Deleted Scenes', 'Behind the Scenes']",
 "['Trailers', 'Deleted Scenes', 'Behind the Scenes']",
 "['Commentaries']",
 "['Behind the Scenes']",
 "['Deleted Scenes', 'Behind the Scenes']",
 "['Commentaries', 'Behind the Scenes']",
 "['Trailers', 'Behind the Scenes']",
 "['Trailers', 'Commentaries', 'Behind the Scenes']",
 "['Deleted Scenes']",
 "['Commentaries', 'Deleted Scenes']",
 "['Trailers', 'Commentaries', 'Deleted Scenes']",
 "['Trailers', 'Deleted Scenes']",
 "['Trailers']",
 "['Trailers', 'Commentaries', 'Deleted Scenes', 'Behind the Scenes']"]

In [66]:
app = dash.Dash(
  __name__
)


app.layout = html.Div([
                html.Div([
                    html.H1('Интерактивные графики для анализа данных ВкусВилл'),
                    
                    html.P('Анализ данных о багах.'
                        'Используйте филтры,чтобы увидеть результат.')],
                    style = {'borderBottom': 'thin lightgrey solid','backgroundColor': 'rgb(250, 250, 250)',
                             'padding': '10px 5px' }) ,
html.Div([
    html.Div([
        html.Label('Фильтр города'),
        dcc.Dropdown(
            id = 'crossfilter-city',
            options = [{'label': i, 'value': i} for i in city],
            value = ['Cheju',
 'Araatuba',
 'Rockford',
 'La Romana',
 'Dadu',
 'Deba Habe',
 'Probolinggo',
 'Jurez',
 'Hsichuh',
 'Huejutla de Reyes',
 'Southend-on-Sea',
 'Daxian',
 'Bchar',
 'Sterling Heights',
 'Springs',
 'Rancagua',
 'Newcastle',
 'Katihar',
 'Cianjur',
 'Atinsk',
 'Bhavnagar',
 'Tanauan',
 'Bratislava',
 'Nukualofa',
 'Ahmadnagar',
 'Karnal',
 'Salinas',
 'Bijapur',
 'Okayama',
 'Coquimbo',
 'Benguela',
 'Sanya',
 'Santiago de Compostela',
 'Jining',
 'Gorontalo',
 'Athenai',
 'Nantou',
 'Kimberley',
 'Tangail',
 'Kuching',
 'Jaipur'],
            multi = True
        )
    ],
    style = {'width': '49%', 'display': 'inline-block'})
]),

html.Div([
    html.Label("Фильтр области"),
    dcc.Dropdown(
        id = 'crossfilter-district',
        options = [{'label': i, 'value': i} for i in district],
        value = ['Cheju',
 'So Paulo',
 'Illinois',
 'La Romana',
 'Sind',
 'Bauchi & Gombe',
 'East Java',
 'Chihuahua',
 'Taipei',
 'Hidalgo',
 'England',
 'Sichuan',
 'Bchar',
 'Michigan',
 'Gauteng',
 'OHiggins',
 'KwaZulu-Natal',
 'Bihar',
 'West Java',
 'Krasnojarsk',
 'Gujarat',
 'Southern Tagalog',
 'Bratislava',
 'Tongatapu',
 'Maharashtra',
 'Haryana',
 'California',
 'Karnataka',
 'Okayama',
 'Coquimbo',
 'Benguela',
 'Hainan',
 'Galicia',
 'Inner Mongolia',
 'Sulawesi Utara',
 'Attika',
 'Nantou',
 'Northern Cape',
 'Dhaka',
 'Sarawak',
 'Rajasthan'],
        multi = True
        )
]),
html.Div([
    html.Label("Категории"),
    dcc.Dropdown(
        id = 'crossfilter-category',
        options = [{'label': i, 'value': i} for i in category],
        value = ['Horror',
 'Foreign',
 'Family',
 'Travel',
 'Sci-Fi',
 'Animation',
 'Drama',
 'New',
 'Music',
 'Classics',
 'Action',
 'Documentary',
 'Comedy',
 'Children',
 'Games',
 'Sports'],
        multi = True
    )
]),
html.Div([
    html.Label("Рейтинг"),
    dcc.Dropdown(
        id = 'crossfilter-rating',
        options = [{'label': i, 'value': i} for i in rating],
        value = ['PG-13', 'NC-17', 'G', 'PG', 'R'],
        multi = True
    )
]),
html.Div([
    html.Label("Название фильма"),
    dcc.Dropdown(
        id = 'crossfilter-title',
        options = [{'label': i, 'value': i} for i in title],
        value = [],
        multi = True
    )
]),
html.Div([
    html.Label("Фамилия работника"),
    dcc.Dropdown(
        id = 'crossfilter-last_name',
        options = [{'label': i, 'value': i} for i in last_name],
        value = ['Stephens', 'Hillyer'],
        multi = True
    )
]),
html.Div([
    html.Label("Язык фильма"),
    dcc.Dropdown(
        id = 'crossfilter-name',
        options = [{'label': i, 'value': i} for i in name],
        value = [],
        multi = True
    )
]),
html.Div([
    html.Label("Год выпуска"),
    dcc.Dropdown(
        id = 'crossfilter-release_year',
        options = [{'label': i, 'value': i} for i in release_year],
        value = [2006],
        multi = True
    )
]),
html.Div([
    html.Label("Особенности фильма"),
    dcc.Dropdown(
        id = 'crossfilter-special_features',
        options = [{'label': i, 'value': i} for i in special_features],
        value = ["['Trailers', 'Commentaries']",
 "['Commentaries', 'Deleted Scenes', 'Behind the Scenes']",
 "['Trailers', 'Deleted Scenes', 'Behind the Scenes']",
 "['Commentaries']",
 "['Behind the Scenes']",
 "['Deleted Scenes', 'Behind the Scenes']",
 "['Commentaries', 'Behind the Scenes']",
 "['Trailers', 'Behind the Scenes']",
 "['Trailers', 'Commentaries', 'Behind the Scenes']",
 "['Deleted Scenes']",
 "['Commentaries', 'Deleted Scenes']",
 "['Trailers', 'Commentaries', 'Deleted Scenes']",
 "['Trailers', 'Deleted Scenes']",
 "['Trailers']",
 "['Trailers', 'Commentaries', 'Deleted Scenes', 'Behind the Scenes']"],
        multi = True
    )
]),
html.Div(
    dcc.Slider(
        id = 'crossfilter-length',
        min = df['length'].min(),
        max = df['length'].max(),
        
        value = 105,
        step = None,
        marks = {str(length):
                 str(length) for length in df['length'].unique()}
                ),
    style = {'width': '230%', 'padding': '0px 20px 20px 20px'}
),  
    
    
html.Div(
    dcc.Graph(id = 'histogram'),
    style = {'width': '101%', 'display': 'inline-block'}
),
html.Div(
    dcc.Graph(id = 'scatter-plot'),
    style = {'width': '121%', 'display': 'inline-block'}
),

    html.Div(
    dcc.Graph(id = 'histogram-2'),
    style = {'width': '101%', 'display': 'inline-block','stackgroup':'one'}
    
),
     html.Div(
    dcc.Graph(id = 'histogram-3'),
    style = {'width': '101%', 'display': 'inline-block','stackgroup':'one'}
    
),
    html.Div(
    dcc.Graph(id = 'ecdf'),
    style = {'width': '101%', 'display': 'inline-block','stackgroup':'one'}
)
])    

@app.callback(
    Output('histogram','figure'),
    [Input('crossfilter-city','value'),
    Input('crossfilter-district','value'),
    Input('crossfilter-category','value'),
    Input('crossfilter-rating','value'),
    Input('crossfilter-length','value')])
def update_stacked_area(city,district,category,rating,length):
    
    filtered_data = df[(df['city'].isin(city))&(df['district'].isin(district))&
                      (df['category'].isin(category))&(df['rating'].isin(rating))
                      &(df['length']<=length)]
    figure_hist = px.histogram(filtered_data,
        x = "category",y='amount', color = "rating",
        title = "Распредление выручки по категории и рейтингу")
    figure_hist.update_layout(yaxis_title = "Сумма выручки")
    return figure_hist
    

@app.callback(
    Output('histogram-2','figure'),
    [Input('crossfilter-city','value'),
    Input('crossfilter-district','value'),
    Input('crossfilter-category','value'),
    Input('crossfilter-rating','value'),
    Input('crossfilter-length','value')])
def update_stacked_area(city,district,category,rating,length):
    
    filtered_data = df[(df['city'].isin(city))&(df['district'].isin(district))&
                      (df['category'].isin(category))&(df['rating'].isin(rating))
                      &(df['length']<=length)]
    figure_hist = px.histogram(filtered_data,
        x = "category", color = "rating",
        title = "Распредление количества заказов по категории и рейтингу")
    figure_hist.update_layout(yaxis_title = "Кол-во")
    return figure_hist


@app.callback(
    Output('scatter-plot','figure'),
    [Input('crossfilter-city','value'),
    Input('crossfilter-district','value'),
    Input('crossfilter-category','value'),
    Input('crossfilter-rating','value')])
def update_stacked_area(city,district,category,rating):
    
    filtered_data = df[(df['city'].isin(city))&(df['district'].isin(district))&
                      (df['category'].isin(category))&(df['rating'].isin(rating))
        ]  
    
    figure = px.scatter(filtered_data,x='rental_period',y='length', color = "rating",size="amount",
        title = "Соотношение выручки к продолжительности фильма в разбиении по рейтингу")
    figure.update_layout(yaxis_title = "выручка")
    return figure    

@app.callback(
    Output('histogram-3','figure'),
    [Input('crossfilter-city','value'),
    Input('crossfilter-district','value'),
    Input('crossfilter-category','value'),
    Input('crossfilter-rating','value'),
    Input('crossfilter-release_year','value'),
    Input('crossfilter-special_features','value'),
    Input('crossfilter-length','value')])
def update_stacked_area(city,district,category,rating,release_year,special_features,length):
    
    filtered_data = df[(df['city'].isin(city))&(df['district'].isin(district))&
                      (df['category'].isin(category))&(df['rating'].isin(rating))&
                      (df['special_features'].isin(special_features))&(df['release_year'].isin(release_year))
                      &(df['length']<=length)]
    figure_hist = px.histogram(filtered_data,y='amount',marginal="rug", color = "last_name",
        title = "Распредление выручки по сотрудникам")
    figure_hist.update_layout()
    return figure_hist

@app.callback(
    Output('ecdf','figure'),
    [Input('crossfilter-city','value'),
    Input('crossfilter-district','value'),
    Input('crossfilter-category','value'),
    Input('crossfilter-rating','value'),
    Input('crossfilter-release_year','value'),
    Input('crossfilter-special_features','value'),
    Input('crossfilter-length','value'),
    Input('crossfilter-title','value'),
    Input('crossfilter-last_name','value'),
    Input('crossfilter-name','value')])
def update_stacked_area(city,district,category,rating,release_year,
                        special_features,length,name,last_name,title):
    
    filtered_data = df[(df['city'].isin(city))&(df['district'].isin(district))&
                      (df['category'].isin(category))&(df['rating'].isin(rating))&
                      (df['special_features'].isin(special_features))&(df['release_year'].isin(release_year))&
                      (df['last_name'].isin(last_name))&(df['title'].isin(title))
                      &(df['length']<=length)]
    figure_hist = px.ecdf(filtered_data,x='amount', color = "rating",
        title = "Распределение стоимости фильма по рейтинговым категориям")
    figure_hist.update_layout()
    return figure_hist

if __name__ == '__main__':
    app.run_server(debug=True, port=1213)
